# ETL 5 Pair 25/08
### ETL Carga I

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [83]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import mysql.connector

from datetime import datetime, timedelta

In [84]:
df_energia_ccaa = pd.read_pickle("data/pairETL3_energia_ccaa.pkl")
df_energia_ccaa.head(2)

,value,percentage,tipo_energia,comunidad,id_comunidad,fecha
0,460.05,1.0,No renovable,Ceuta,8744,2011-01-01
1,462.98,1.0,No renovable,Ceuta,8744,2011-01-02


In [85]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

   
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
      
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    def check_comunidades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_ciudad = f"""
                SELECT DISTINCT comunidad FROM comunidades
                """
        mycursor.execute(query_existe_ciudad)
        comunidades = mycursor.fetchall()
        return comunidades
    
    # método para sacar el id de una ciudad en concreto 
    def sacar_id_comunidad(self, comunidad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidad FROM comunidades WHERE comunidad = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. "
     
    
    def sacar_id_fecha(self, fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idtiempo FROM fechas WHERE fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "
            

In [86]:
tabla_fechas = '''
CREATE TABLE IF NOT EXISTS `energia`.`fechas` (
  `idfechas` INT NOT NULL,
  `fecha` DATE NULL,
  PRIMARY KEY (`idfechas`))
ENGINE = InnoDB;
'''

In [87]:
tabla_nacional_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_fechas_idx` (`fechas_idfechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_fechas`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;
'''

In [88]:
tabla_comunidades = '''
CREATE TABLE IF NOT EXISTS `energia`.`comunidades` (
  `idcomunidades` INT NOT NULL,
  `comunidades` VARCHAR(45) NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
'''

In [89]:
tabla_comunidades_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_idfechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `energia`.`comunidades` (`idcomunidades`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;
'''

In [90]:
carga = Cargar("energia", "AlumnaAdalab")

In [91]:
carga

In [92]:
carga.crear_bbdd()

Conexión realizada con éxito


In [93]:
carga.crear_insertar_tabla(tabla_fechas)
carga.crear_insertar_tabla(tabla_nacional_renovable_no_renovable)
carga.crear_insertar_tabla(tabla_comunidades)
carga.crear_insertar_tabla(tabla_comunidades_renovable_no_renovable)

In [94]:
id_comunidad = carga.sacar_id_comunidad("Madrid")
id_comunidad

'Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. '

In [95]:
id_fecha = carga.sacar_id_fecha(2011-11-10)
id_fecha

'Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. '

In [96]:
df_energia_ccaa[["fecha"]].head()

,fecha
0,2011-01-01
1,2011-01-02
2,2011-01-03
3,2011-01-04
4,2011-01-05


In [97]:
for indice, fila in df.iterrows(): 
    
    query_tiempo = f"""
            INSERT INTO tiempo (fecha) 
            VALUES ("{fila['fecha']}");
            """
    carga.crear_insertar_tabla(query_tiempo)
    
    break

1146 (42S02): Table 'energia.tiempo' doesn't exist
Error Code: 1146
SQLSTATE 42S02
Message Table 'energia.tiempo' doesn't exist


In [98]:
df_energia_ccaa[['comunidad']].head()

,comunidad
0,Ceuta
1,Ceuta
2,Ceuta
3,Ceuta
4,Ceuta


In [99]:
# NO ME SALE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidad = f"""
                INSERT INTO comunidades (comunidades) 
                VALUES ("{fila['comunidades']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    comunidades = carga.check_comunidades()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(comunidades) == 0 or fila['comunidades'] not in comunidades[0]: 
        carga.crear_insertar_tabla( query_comunidad)

    else:
        print(f"{fila['comunidades']} ya esta en nuestra BBDD")

KeyError: 'comunidades'

In [ ]:
for indice, fila in df.iterrows():
    
    id_comunidad = carga.sacar_id_comunidad(fila['idcomunidades_renovable_no_renovable'])
    #id_fecha = carga.sacar_id_fecha(fila["fechas_idfechas"])
    
    query_valores_ccaa = f"""
                INSERT INTO comunidades_renovable_no_renovable (idcomunidades_renovable_no_renovable, porcentaje, tipo_energia, valor, fechas_idfechas, comunidades_idcomunidades) 
                VALUES ({fila["idcomunidades_renovable_no_renovable"]}, {fila['porcentaje']}, "{fila['tipo_energia']}", {fila['valor']}, {idfechas}, {idcomunidades});
                """
    carga.crear_insertar_tabla(query_valores_ccaa)
    

In [ ]:
for indice, fila in df.iterrows():
    
    id_nacional = carga.sacar_id_nacional(fila['idnacional_renovable_no_renovable'])
    #id_fecha = carga.sacar_id_fecha(fila["fechas_idfechas"])
    
    query_valores_nacional = f"""
                INSERT INTO nacional_renovable_no_renovable (idnacional_renovable_no_renovable, porcentaje, tipo_energia, valor, fechas_idfechas) 
                VALUES ({fila["idnacional_renovable_no_renovable"]}, {fila['porcentaje']}, "{fila['tipo_energia']}", {fila['valor']}, {idfechas});
                """
    carga.crear_insertar_tabla(query_valores_nacional)
    